# Functions as First class Citizens

In [2]:
x = 100


def foo(y):
    return x + y


z = foo(307)
print(x, z, foo)


def bar(x):
    x = 1000
    return foo(308)


w = bar(349)
print(x, w)


def apply_n_times(f, n, x):
    out = x
    for i in range(n):
        out = f(out)

    return out


def double(x):
    return x * 2


def outer(func):
    def inner():
        print("Hi")

    return inner


@outer
def sayHi():
    print("Hello")


sayHi()

# Decorators and Closures

In [15]:
def decorate(func):
    def inner():
        print("Inner Function")
        return 2 * func()  # Know little this level of python

    return inner


"""
Decorators first execute their function, then lazily execute the function they are called with and finally calculate the result of decorated function
"""


@decorate
def target():
    print("Target Function")
    return 2


# print(decorate(target()))
print("Target:", target())

In [27]:
"""
Calculate double of a function 
"""


def double_func(func):
    def inner(x):  # x is passed by the target function to the decorated function
        print(x)
        return 2 * func(x)

    return inner


@double_func
def half_of_x(x):
    x = x // 2
    print(x)
    return x


print(half_of_x(3))  # Apparently 2 is being passed

In [28]:
registry = []


def register(func):
    print(f"running register({func})")
    registry.append(func)
    return func


@register
def foo1():
    print("foo1 Function")


@register
def foo2():
    print("foo2 Function")


def foo3():
    print("foo3 Function")

In [29]:
print("Invoking Register", registry)

In [30]:
print(decorate(target))

In [31]:
print(register(foo1))

# Note that register runs (twice) before any other function in the module. When register is called, it receives the decorated function object as an argument—

for example, <function f1 at 0x100631bf8>.

In [32]:
for reg in registry:
    reg()

In [33]:
foo1()

In [34]:
registry[1]

In [35]:
registry[0]()

In [36]:
for reg in registry:
    reg()

In [40]:
import Register

In [41]:
Register.registry

# Closures

In [89]:
class Averager:
    """
    Starter for closures and OOPS
    variable with _ has class scope and __ is a private attribute
    """

    def __init__(self):

        self._series = []
        self.__series1 = []

    def __call__(self, val):
        self._series.append(val)
        self.__series1.append(val)
        total = sum(self._series) / len(self._series)

        return total

In [90]:
avg = Averager()
print(avg(10))
print(avg(100))

In [93]:
print(avg._series)
# So wondering why bother with private static protected and public variables in Java

In [92]:
avg.__series1

In [98]:
"""
Functional Way
Closure
"""


def make_average():
    series = []

    def calculate_average(num):
        series.append(num)
        total = sum(series) / len(series)
        return total

    return calculate_average

In [99]:
avg = make_average()
print(avg(10))

In [100]:
avg(11)

In [101]:
avg(12)

In [102]:
avg.__closure__[0].cell_contents

In [114]:
avg.__code__.co_freevars

In [7]:
from dis import dis

In [104]:
dis(foo1)

In [106]:
dis(make_average)

In [42]:
def generate_parenthesis(n):
    series = []

    def dfs(left, right, s):
        if left + right == 2 * n:
            series.append(s)
            return

        if left < n:
            dfs(left + 1, right, s + "(")
        if right < left:
            dfs(left, right + 1, s + ")")

    dfs(0, 0, "")
    return series

In [43]:
print(generate_parenthesis(4))

In [109]:
gen = generate_parenthesis(4)

In [14]:
dis(generate_parenthesis)  # Without Return

In [6]:
%timeit generate_parenthesis(4)

In [115]:
gen[0].__code__.co_freevars

In [16]:
dis(generate_parenthesis)

# Non Locals

In [17]:
def calulate_average():
    count = 0
    total = 0

    def averager(val):
        nonlocal count, total
        count += 1
        total += val
        return total / count

    return averager

In [18]:
x = calulate_average()
x(10)
print(x(11))
print(x.__closure__[0].cell_contents)

In [54]:
list1 = [1, 2, 3, 4, 5]
list2 = ["a", "b", "c", "d"]
pairs = [pair for pair in zip(list1, list2)]

In [55]:
letters, numbers = zip(*pairs)  # Unzip
print(letters, numbers)

In [58]:
def doubler(f):
    def g(x):
        return 2 * f(x)

    return g


def f1(x):
    return x + 1


a = doubler(f1)
assert a(4) == 10

In [44]:
import time

DEFAULT_FMT = "[{elapsed:0.8f}s] {name}({args}) -> {result}"


def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*args):
            start = time.perf_counter()
            _result = func(*args)
            elapsed = time.perf_counter() - start
            name = func.__name__
            args = ",".join(repr(arg) for arg in args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result

        return clocked

    return decorate

In [64]:
@clock()
def snooze(seconds):
    time.sleep(seconds)


for i in range(3):
    snooze(0.123)

In [65]:
@clock("{name}:{elapsed}s")
def snooze_again(seconds):
    time.sleep(seconds)


for i in range(3):
    snooze_again(0.123)

In [46]:
print(clock(generate_parenthesis(4)))

In [1]:
import time
import functools


def clock1(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = (time.perf_counter() - t0) * 100000
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f"{k}={v!r}" for k, v in kwargs.items())
        arg_str = ", ".join(arg_lst)
        print(f"[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}")
        return result

    return clocked

In [50]:
print(clock1(generate_parenthesis(4)))

In [2]:
@clock1
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)

In [3]:
print(factorial(45))

In [6]:
@clock1
def generate_parenthesis_with_return(n):
    series = []

    def dfs(left, right, s):
        if left + right == 2 * n:
            series.append(s)
            return

        if left < n:
            dfs(left + 1, right, s + "(")
        if right < left:
            dfs(left, right + 1, s + ")")

    dfs(0, 0, "")
    return series

In [ ]:
print(generate_parenthesis_with_return(10))

In [65]:
@clock1
def generate_parenthesis_without_return(n):
    series = []

    def dfs(left, right, s):
        if left + right == 2 * n:
            series.append(s)

        if left < n:
            dfs(left + 1, right, s + "(")
        if right < left:
            dfs(left, right + 1, s + ")")

    dfs(0, 0, "")
    return series

In [66]:
print(generate_parenthesis_without_return(5))

In [4]:
@clock1
def generate_parenthesis_using_yield(n):
    def dfs(left, right, s):
        if left + right == 2 * n:
            yield s
        if left < n:
            dfs(left + 1, right, s + "(")
        if right < left:
            dfs(left, right + 1, s + ")")

    return list(dfs(0, 0, ""))

In [5]:
print(generate_parenthesis_using_yield(3))

In [ ]:
from __future__ import annotations


def tokenise(text: str) -> list[str]:
    return text.upper().split()